In [1]:
import pandas as pd

In [2]:
courier_company_rates = pd.read_csv('Courier Company - Rates.csv')
invoice = pd.read_csv('Invoice.csv')
order_report = pd.read_csv('Order Report.csv')
pincodes = pd.read_csv('pincodes.csv')
sku_master = pd.read_csv('SKU Master.csv')

In [6]:
print("Order Report: ")
print(order_report.head())
print("\nSKU Master")
print(sku_master.head())
print("\nInvoice:")
print(invoice.head())
print("\nCourier Company Rates:")
print(courier_company_rates.head())
print("\nPincodes:")
print(pincodes.head())

Order Report: 
   ExternOrderNo            SKU  Order Qty  Unnamed: 3  Unnamed: 4
0     2001827036  8904223818706        1.0         NaN         NaN
1     2001827036  8904223819093        1.0         NaN         NaN
2     2001827036  8904223819109        1.0         NaN         NaN
3     2001827036  8904223818430        1.0         NaN         NaN
4     2001827036  8904223819277        1.0         NaN         NaN

SKU Master
             SKU  Weight (g)  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  8904223815682         210         NaN         NaN         NaN
1  8904223815859         165         NaN         NaN         NaN
2  8904223815866         113         NaN         NaN         NaN
3  8904223815873          65         NaN         NaN         NaN
4  8904223816214         120         NaN         NaN         NaN

Invoice:
        AWB Code    Order ID  Charged Weight  Warehouse Pincode  \
0  1091117222124  2001806232            1.30             121003   
1  1091117222194  2001806273         

In [8]:
#Check for missing Values
print("Missing Value in Order Report: ")
print(order_report.isnull().sum())
print("\nMissing Value in SKU Master: ")
print(sku_master.isnull().sum())
print("\nMissing Value in Invoice: ")
print(invoice.isnull().sum())
print("\nMissing Value in Courier Company Rates: ")
print(courier_company_rates.isnull().sum())
print("\nMissing Value in Pincode Data: ")
print(pincodes.isnull().sum())

Missing Value in Order Report: 
ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64

Missing Value in SKU Master: 
SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64

Missing Value in Invoice: 
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

Missing Value in Courier Company Rates: 
fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_additional    0
fwd_e_fixed         0
fwd_e_additional    0
rto_a_fixed         0
rto_a_additional    0
rto_b_fixed         0
rto_b_additional    0
rto_c_fixed         0
rto_c_additional    0
rto_d_fixed         0
rto_d_additional    0
rto_e_fixed         0
rto_e_additi

In [9]:
## Remove the unnamed columns
order_report = order_report.drop(columns = ['Unnamed: 3','Unnamed: 4'])
sku_master = sku_master.drop(columns = ['Unnamed: 2','Unnamed: 3','Unnamed: 4'])
pincodes = pincodes.drop(columns= ['Unnamed: 3','Unnamed: 4'])

In [11]:
## Merge the order_report and sku_master through sku 
merged_data = pd.merge(order_report,sku_master,on='SKU')
print(merged_data.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)
0     2001827036  8904223818706        1.0         127
1     2001827036  8904223819093        1.0         150
2     2001827036  8904223819109        1.0         100
3     2001827036  8904223818430        1.0         165
4     2001827036  8904223819277        1.0         350


In [14]:
merged_data = merged_data.rename(columns={'ExternOrderNo':'Order ID'})

In [15]:
abc_courier = pincodes.drop_duplicates(subset = ['Customer Pincode'])
courier_abc = invoice[['Order ID','Customer Pincode','Type of Shipment']]
pincode = courier_abc.merge(abc_courier, on = 'Customer Pincode')
print(pincode.head())

     Order ID  Customer Pincode Type of Shipment  Warehouse Pincode Zone
0  2001806232            507101  Forward charges             121003    d
1  2001806273            486886  Forward charges             121003    d
2  2001806408            532484  Forward charges             121003    d
3  2001806458            143001  Forward charges             121003    b
4  2001807012            515591  Forward charges             121003    d


In [16]:
merged2 = merged_data.merge(pincode,on='Order ID')

In [17]:
print(merged2.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Zone  
0  Forward charges             121003    e  
1  Forward charges             121003    e  
2  Forward charges             121003    e  
3  Forward charges             121003    e  
4  Forward charges             121003    e  


In [18]:
## Weight in KG
merged2['Weights (kgs)'] =merged2['Weight (g)']/1000 

In [27]:
def weight_slab(weight):
    i = round(weight%1,1)
    if i == 0.0:
        return weight
    elif i>0.5:
        return int(weight)+1.0
    else:
        return int(weight)+0.5
merged2['Weight Slab (KG)'] = merged2['Weights (kgs)'].apply(weight_slab)
invoice['Weight Slab Changed by Courier Comapny'] = (invoice['Charged Weight'].apply(weight_slab))

In [28]:
invoice = invoice.rename(columns={'Zone': 'Delivery Zone Charged by Courier Company'})
merged2 = merged2.rename(columns={'Zone': 'Delivery Zone As Per ABC'})
merged2 = merged2.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per ABC'})

In [30]:
merged2.head(15)

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per ABC,Weights (kgs),Weight Slab As Per ABC
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.50
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.50
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.50
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.50
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.50
5,2001827036,GIFTBOX202002,1.0,500,173213,Forward charges,121003,e,0.500,0.50
6,2001827036,GIFTBOX202002,1.0,500,173213,Forward charges,121003,e,0.500,0.50
7,2001827036,8904223818638,2.0,137,173213,Forward charges,121003,e,0.137,0.50
8,2001827036,SACHETS001,1.0,10,173213,Forward charges,121003,e,0.010,0.01
9,2001825261,8904223819024,4.0,112,517128,Forward and RTO charges,121003,d,0.112,0.50


In [33]:
## Calculate the Expected Charges
total_expected_charge = []

for _, row in merged2.iterrows():
    fwd_category = 'fwd_' + row['Delivery Zone As Per ABC']
    fwd_fixed = courier_company_rates.at[0, fwd_category + '_fixed']
    fwd_additional = courier_company_rates.at[0, fwd_category + '_additional']
    rto_category = 'rto_' + row['Delivery Zone As Per ABC']
    rto_fixed = courier_company_rates.at[0, rto_category + '_fixed']
    rto_additional = courier_company_rates.at[0, rto_category + '_additional']

    weight_slab = row['Weight Slab As Per ABC']

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_expected_charge.append(0)

merged2['Expected Charge as per ABC'] = total_expected_charge
merged2.head(20)

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per ABC,Weights (kgs),Weight Slab As Per ABC,Expected Charge as per ABC
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.50,56.6
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.50,56.6
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.50,56.6
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.50,56.6
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.50,56.6
5,2001827036,GIFTBOX202002,1.0,500,173213,Forward charges,121003,e,0.500,0.50,56.6
6,2001827036,GIFTBOX202002,1.0,500,173213,Forward charges,121003,e,0.500,0.50,56.6
7,2001827036,8904223818638,2.0,137,173213,Forward charges,121003,e,0.137,0.50,56.6
8,2001827036,SACHETS001,1.0,10,173213,Forward charges,121003,e,0.010,0.01,56.6
9,2001825261,8904223819024,4.0,112,517128,Forward and RTO charges,121003,d,0.112,0.50,45.4


In [34]:
merged_output = merged2.merge(invoice, on='Order ID')
merged_output.head(20)

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode_x,Type of Shipment_x,Warehouse Pincode_x,Delivery Zone As Per ABC,Weights (kgs),Weight Slab As Per ABC,Expected Charge as per ABC,AWB Code,Charged Weight,Warehouse Pincode_y,Customer Pincode_y,Delivery Zone Charged by Courier Company,Type of Shipment_y,Billing Amount (Rs.),Weight Slab Changed by Courier Comapny
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.50,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.50,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.50,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.50,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.50,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
5,2001827036,GIFTBOX202002,1.0,500,173213,Forward charges,121003,e,0.500,0.50,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
6,2001827036,GIFTBOX202002,1.0,500,173213,Forward charges,121003,e,0.500,0.50,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
7,2001827036,8904223818638,2.0,137,173213,Forward charges,121003,e,0.137,0.50,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
8,2001827036,SACHETS001,1.0,10,173213,Forward charges,121003,e,0.010,0.01,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
9,2001825261,8904223819024,4.0,112,517128,Forward and RTO charges,121003,d,0.112,0.50,45.4,1091121981575,1.6,121003,517128,d,Forward and RTO charges,345.0,2.0


In [35]:
## Difference in Charges
df_diff = merged_output
df_diff['Difference (Rs.)'] = df_diff['Billing Amount (Rs.)'] - df_diff['Expected Charge as per ABC']

df_new = df_diff[['Order ID', 'Difference (Rs.)', 'Expected Charge as per ABC']]

df_new.head()

,Order ID,Difference (Rs.),Expected Charge as per ABC
0,2001827036,61.3,56.6
1,2001827036,61.3,56.6
2,2001827036,61.3,56.6
3,2001827036,61.3,56.6
4,2001827036,61.3,56.6


In [38]:
## Summarize the Accuracy
total_correctly_charged = len(df_new[df_new['Difference (Rs.)'] == 0])
total_overcharged = len(df_new[df_new['Difference (Rs.)'] > 0])
total_undercharged = len(df_new[df_new['Difference (Rs.)'] < 0])

amount_overcharged = abs(df_new[df_new['Difference (Rs.)'] > 0]['Difference (Rs.)'].sum())
amount_undercharged = df_new[df_new['Difference (Rs.)'] < 0]['Difference (Rs.)'].sum()
amount_correctly_charged = df_new[df_new['Difference (Rs.)'] == 0]['Expected Charge as per ABC'].sum()

summary_data = {'Description': ['Total Orders where ABC has been correctly charged',
                                'Total Orders where ABC has been overcharged',
                                'Total Orders where ABC has been undercharged'],
                'Count': [total_correctly_charged, total_overcharged, total_undercharged],
                'Amount (Rs.)': [amount_correctly_charged, amount_overcharged, amount_undercharged]}

df_summary = pd.DataFrame(summary_data)
df_summary

,Description,Count,Amount (Rs.)
0,Total Orders where ABC has been correctly charged,12,507.6
1,Total Orders where ABC has been overcharged,382,33750.5
2,Total Orders where ABC has been undercharged,7,-165.2


In [40]:
import plotly.graph_object as go

ModuleNotFoundError: No module named 'plotly.graph_object'

In [39]:
## Plot the Error

fig = go.figure(data= go.pie(labels = df_summary['Description'],
                            values =df_summary['Count'],
                            textinfo = 'label+percent',
                            hole = 0.4,
                            title = 'Proportion'))
fig.show()


ModuleNotFoundError: No module named 'plotly.graph_object'